In [ ]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

#generator = keras.models.load_model(f"./models/sixty_seconds/cond_generator_2806_0.65625")

generator = keras.models.load_model(f"./models/no_dp/1000/cgan_generator")
generator_dp = keras.models.load_model(f"./models/dp/1000/cgan_generator")

# discriminator = keras.models.load_model("./models/cond_discriminator")

In [ ]:
num_syn_samples = 36 // 2 * 150
latent_dim = 60

label_no_stress = tf.zeros([num_syn_samples,1])
label_stress = tf.ones([num_syn_samples,1])

random_vector = tf.random.normal(shape=(num_syn_samples,latent_dim))

syn_no_stress = generator([random_vector, label_no_stress])
syn_stress = generator([random_vector, label_stress])

dp_syn_no_stress = generator_dp([random_vector, label_no_stress])
dp_syn_stress = generator_dp([random_vector, label_stress])

zero = np.zeros([num_syn_samples, 60, 1])
ones = np.ones([num_syn_samples, 60, 1])

no_stress = np.append(np.array(syn_no_stress), zero, axis=2)
stress = np.append(np.array(syn_stress), ones, axis=2)

dp_no_stress = np.append(np.array(dp_syn_no_stress), zero, axis=2)
dp_stress = np.append(np.array(dp_syn_stress), ones, axis=2)

gen_data = np.concatenate((no_stress, stress))

dp_gen_data = np.concatenate((dp_no_stress, dp_stress))

In [ ]:
# save dp synthetic data
with open (f"data/syn/cgan/dp/5400.npy", "wb") as f:
    np.save(f, dp_gen_data)
# save synthetic data
with open (f"data/syn/cgan/no_dp/5400.npy", "wb") as f:
    np.save(f, gen_data)